In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
len(words)

32033

In [8]:
# build vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [9]:
# build the dataset

block_size = 3 # context length: how many characters do we use to predict the next one?
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [11]:
C = torch.randn((27,2))
# two ways of embedding a certain number
print(f"Normal direct access: {C[5]}")
print(f"Matrix Mul: {F.one_hot(torch.tensor(5), num_classes=27).float() @ C}")

Normal direct access: tensor([-0.3335, -0.7040])
Matrix Mul: tensor([-0.3335, -0.7040])


In [17]:
# we can embed the whole vector at once directly
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [18]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [24]:
# we want to squash two dimensions to have a 32x6 tensor, so we can multiply with 6x100 weights
# emb = torch.cat(torch.unbind(emb, 1), 1)
# emb.shape

# A more efficient way of doing this is by using .view()
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h.shape

torch.Size([32, 100])

In [25]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [26]:
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [28]:
counts = logits.exp()

In [29]:
probs = counts / counts.sum(1, keepdims=True)

In [31]:
probs.shape

torch.Size([32, 27])

In [34]:
loss = -probs[torch.arange(32), Y].log().mean()
loss

tensor(15.2756)

In [35]:
parameters = [C, W1, b1, W2, b2]
sum(p.nelement() for p in parameters)

3481

In [36]:
# More organizedly (and efficient):
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2 # (32, 27)
# counts = logits.exp()
# probs = counts / counts.sum(1, keepdims=True)
# loss = -probs[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits, Y) # much more efficient
loss

tensor(15.2756)